In [1]:
from arango import ArangoClient

ArangoClient = ArangoClient(hosts="http://127.0.0.1:8530")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

collections = db.collections()


In [6]:
import json

def preprocess_json(input_file, output_file):
    with open(input_file, 'r') as f:
        # Read the entire file content
        data = f.read()

        # Split the content by newline character to handle multiple JSON objects
        json_objects = data.strip().split('\n')

    # Process each JSON object separately
    processed_data = []
    for json_str in json_objects:
        try:
            # Load each JSON object separately
            obj = json.loads(json_str)
            processed_data.append(obj)
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

    # Write processed data to the output file
    with open(output_file, 'w') as f:
        json.dump(processed_data, f)

if __name__ == "__main__":
    input_file = "yelp_academic_dataset_user.json"  # Replace with the path to your input JSON file
    output_file = "yelp_academic_dataset_user_updated.json"  # Replace with the path to your output JSON file
    preprocess_json(input_file, output_file)


In [7]:
import json
collection_name = "user"
collection=db.collection(collection_name)
# Path to the JSON file
json_file = "yelp_academic_dataset_user_updated.json"  

# Read the JSON data from file
with open(json_file, 'r') as f:
    data = json.load(f)

# Bulk insert the data into the collection
collection.import_bulk(data)

ConnectionAbortedError: Can't connect to host(s) within limit (3)